In [1]:
import Pkg
Pkg.activate("../../tensor")
import tensor as ten

  Activating project at `c:\Users\kakin\home\workspace\tensor`


In [2]:
using ITensors


#=テスト
# インデックス定義（2値）
i = Index(2, "i")
j = Index(2, "j")
k = Index(2, "k")
l = Index(2, "l")
m = Index(2, "m")
n = Index(2, "n")

typeof(i)|>display

# テンソル初期化
A = ITensor(i, j, k, l)
B = ITensor(j, i, m, n)

K_const = 1.0

# 関数定義(a,b,c,dは0or1 Kは逆温度を含む定数)
function four_leg_tensor_cal(a::Int64, b::Int64, c::Int64, d::Int64, K::Float64)
    # 0 or 1 のビットを ±1 に変換し、隣接ペアの積を計算
    s1 = 2a - 1
    s2 = 2b - 1
    s3 = 2c - 1
    s4 = 2d - 1
    exponent = K * (s1*s2 + s2*s3 + s3*s4 + s4*s1)
    return exp(exponent)
end

# 代入ループ（0/1ビット→ITensorの1/2インデックスに対応させる）
for a in 0:1, b in 0:1, c in 0:1, d in 0:1
    val = four_leg_tensor_cal(a, b, c, d, K_const)
    A[i => a+1, j => b+1, k => c+1, l => d+1] = val
    B[j => a+1, i => b+1, m => c+1, n => d+1] = val
end

C=A*B

Cmat=Array(C,[k,l,m,n])

Base.print_array(stdout,Cmat)
=#

In [3]:
#Index
#大きな足
α = Index(2, "α")
ξ = Index(2, "ξ")
β = Index(2, "β")
η = Index(2, "η")

#小さな足
i = Index(2, "i")
j = Index(2, "j")
k = Index(2, "k")
l = Index(2, "l")

(dim=2|id=551|"l")

In [4]:
# 関数定義(a,b,c,dは0or1 Kは逆温度を含む定数)
function four_leg_tensor_cal(a::Int64, b::Int64, c::Int64, d::Int64, K::Float64)
    # 0 or 1 のビットを ±1 に変換し、隣接ペアの積を計算
    s1 = 2a - 1
    s2 = 2b - 1
    s3 = 2c - 1
    s4 = 2d - 1
    exponent = K * (s1*s2 + s2*s3 + s3*s4 + s4*s1)
    return exp(exponent)
end

four_leg_tensor_cal (generic function with 1 method)

In [5]:
function four_leg_tensor_def(i::Index{Int64}, j::Index{Int64}, k::Index{Int64}, l::Index{Int64},K_const::Float64)
    W = ITensor(i, j, k, l)
    for a in 0:1, b in 0:1, c in 0:1, d in 0:1
        val = four_leg_tensor_cal(a, b, c, d, K_const)
        W[i => a+1, j => b+1, k => c+1, l => d+1] = val
    end
    return W
end

K_const=1.0

#4脚テンソルの定義
W = four_leg_tensor_def(i, j, k, l,K_const)

ITensor ord=4 (dim=2|id=237|"i") (dim=2|id=367|"j") (dim=2|id=773|"k") (dim=2|id=551|"l")
NDTensors.Dense{Float64, Vector{Float64}}

In [6]:
#2脚テンソル
#Indexサイズを固定端の場合⇒1、自由端の場合⇒2
c1 = Index(2, "c1")
c2 = Index(2, "c2")

function two_leg_tensor_def(i::Index{Int64}, j::Index{Int64}, k::Index{Int64}, l::Index{Int64},K_const::Float64)
    C = ITensor(i, j, k, l)
    for a in 0:dim(i)-1, b in 0:dim(j)-1, c in 0:1, d in 0:1
        val = four_leg_tensor_cal(a, b, c, d, K_const)
        C[i => a+1, j => b+1, k => c+1, l => d+1] = val
    end

    # c1 と c2 に関して縮約
    δ_k = delta(k)  # k に関する単位テンソル
    δ_l = delta(l)  # l に関する単位テンソル

    return C * δ_k * δ_l
end

#2脚テンソルの定義
C = two_leg_tensor_def(α, β, c1, c2, K_const)

ITensor ord=2 (dim=2|id=843|"α") (dim=2|id=129|"β")
NDTensors.Dense{Float64, Vector{Float64}}

In [ ]:
#3脚テンソル(横)
#固定端の場合⇒1、自由端の場合⇒2
c1 = Index(2, "c1")

function three_leg_tensor_def(i::Index{Int64}, j::Index{Int64}, k::Index{Int64}, l::Index{Int64},K_const::Float64)
    P = ITensor(i, j, k, l)
    for a in 0:dim(i)-1, b in 0:dim(j)-1, c in 0:dim(k)-1, d in 0:1
        val = four_leg_tensor_cal(a, b, c, d, K_const)
        P[i => a+1, j => b+1, k => c+1, l => d+1] = val
    end

    # c1 と c2 に関して縮約
    δ_l = delta(l)  # l に関する単位テンソル

    return P * δ_l
end

#3脚テンソルの定義
PR = three_leg_tensor_def(α, ξ, l, c1, K_const)


ITensor ord=3 (dim=2|id=843|"α") (dim=2|id=85|"ξ") (dim=2|id=551|"l")
NDTensors.Dense{Float64, Vector{Float64}}

In [8]:
#3脚テンソル(縦)
#固定端の場合⇒1、自由端の場合⇒2
c1 = Index(2, "c1")

#3脚テンソルの定義
PC = three_leg_tensor_def(β, η, k, c1, K_const)

ITensor ord=3 (dim=2|id=129|"β") (dim=2|id=10|"η") (dim=2|id=773|"k")
NDTensors.Dense{Float64, Vector{Float64}}

In [9]:
C2=PR*C*PC*W

ITensor ord=4 (dim=2|id=85|"ξ") (dim=2|id=10|"η") (dim=2|id=237|"i") (dim=2|id=367|"j")
NDTensors.Dense{Float64, Vector{Float64}}

In [10]:
ξi = combiner(ξ, i)
ηj = combiner(η, j)

C2_ext = ξi * C2 * ηj

α = Index(dim(uniqueinds(ξi)[1]), "α")
β = Index(dim(uniqueinds(ηj)[1]), "β")
replaceinds!(C2_ext, uniqueinds(ξi)[1] => α)
replaceinds!(C2_ext, uniqueinds(ηj)[1] => β)

C2=C2_ext

ITensor ord=2 (dim=4|id=88|"α") (dim=4|id=505|"β")
NDTensors.Dense{Float64, Vector{Float64}}

In [11]:
C2[3,1]

186076.5991744225

In [12]:
#=
ξi=combiner(ξ,i)
ξi|>display
ξi_comb=uniqueinds(ξi)[1]
α = Index(dim(ξi_comb), "α")

ηj=combiner(η,j)
ηj|>display
ηj_comb=uniqueinds(ηj)[1]
β = Index(dim(ηj_comb), "β")
=#

In [13]:
#Index
#大きな足
ξ = Index(2^2, "ξ")
η = Index(2^2, "η")

#固定端の場合⇒1、自由端の場合⇒2
c1 = Index(2, "c1")
c2 = Index(2, "c2")

PR = three_leg_tensor_def(α, ξ, l, c1, K_const)
PC = three_leg_tensor_def(β, η, k, c1, K_const)

C3=PR*C2*PC*W

ITensor ord=4 (dim=4|id=117|"ξ") (dim=4|id=593|"η") (dim=2|id=237|"i") (dim=2|id=367|"j")
NDTensors.Dense{Float64, Vector{Float64}}

In [14]:
PR|>display
PC|>display
W|>display
C2|>display

ITensor ord=3 (dim=4|id=88|"α") (dim=4|id=117|"ξ") (dim=2|id=551|"l")
NDTensors.Dense{Float64, Vector{Float64}}

ITensor ord=3 (dim=4|id=505|"β") (dim=4|id=593|"η") (dim=2|id=773|"k")
NDTensors.Dense{Float64, Vector{Float64}}

ITensor ord=4 (dim=2|id=237|"i") (dim=2|id=367|"j") (dim=2|id=773|"k") (dim=2|id=551|"l")
NDTensors.Dense{Float64, Vector{Float64}}

ITensor ord=2 (dim=4|id=88|"α") (dim=4|id=505|"β")
NDTensors.Dense{Float64, Vector{Float64}}

In [15]:
C2
dim(α)

4